In [2]:
!pip freeze 

aiofiles==0.8.0
aiosqlite==0.17.0
alembic==1.7.7
anyio==3.5.0
argon2-cffi @ file:///C:/ci/argon2-cffi_1613037869401/work
asgi-lifespan==1.0.1
asgiref==3.5.0
async-generator @ file:///home/ktietz/src/ci/async_generator_1611927993394/work
asyncpg==0.25.0
atomicwrites==1.4.0
attrs==21.4.0
azure-core==1.24.1
azure-storage-blob==12.12.0
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
beautifulsoup4==4.10.0
bleach @ file:///tmp/build/80754af9/bleach_1628110601003/work
boto3==1.24.4
botocore==1.27.4
brotlipy==0.7.0
bullet==2.2.0
cachetools==4.2.4
certifi==2021.10.8
cffi @ file:///C:/ci/cffi_1636523946399/work
charset-normalizer @ file:///tmp/build/80754af9/charset-normalizer_1630003229654/work
click==8.0.3
cloudpickle==1.6.0
colorama @ file:///tmp/build/80754af9/colorama_1607707115595/work
coloredlogs==15.0.1
commonmark==0.9.1
ConfigArgParse==1.5.3
coolname==1.1.0
coursera-dl==0.11.5
cramjam==2.5.0
croniter==1.3.5
cryptography==37.0.2
cycler==0.11.0
daemoniker==0.2.3
dask==2

In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

c:\Users\admin\.conda\envs\torch\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\admin\.conda\envs\torch\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [16]:
year=2021
month=2

In [17]:
df = read_data(f'../../../data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [18]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [19]:
y_pred.mean()

16.191691679979066

In [22]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [23]:
df_result = df[['ride_id']]
df_result['prediction'] = y_pred

C:\Users\admin\AppData\Local\Temp/ipykernel_15884/4258403066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result['prediction'] = y_pred


In [24]:
output_file = f'predictions_{year:04d}-{month:02d}.parquet'

In [25]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)